In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load preprocessed data
train_images = np.load('../../data/processed/train_images.npy')
train_labels = np.load('../../data/processed/train_labels.npy')
test_images = np.load('../../data/processed/test_images.npy')
test_labels = np.load('../../data/processed/test_labels.npy')

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

In [3]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization


# Define CNN architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    validation_data=(test_images, test_labels),
                    epochs=30,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/30
898/898 [==============================] - 161s 178ms/step - loss: 1.8101 - accuracy: 0.2516 - val_loss: 1.7847 - val_accuracy: 0.2480
Epoch 2/30
898/898 [==============================] - 165s 184ms/step - loss: 1.7766 - accuracy: 0.2603 - val_loss: 1.7133 - val_accuracy: 0.2927
Epoch 3/30
898/898 [==============================] - 157s 175ms/step - loss: 1.7365 - accuracy: 0.2854 - val_loss: 1.6069 - val_accuracy: 0.3672
Epoch 4/30
898/898 [==============================] - 157s 175ms/step - loss: 1.6476 - accuracy: 0.3413 - val_loss: 1.4653 - val_accuracy: 0.4342
Epoch 5/30
898/898 [==============================] - 156s 173ms/step - loss: 1.5614 - accuracy: 0.3863 - val_loss: 1.3994 - val_accuracy: 0.4638
Epoch 6/30
898/898 [==============================] - 157s 174ms/step - loss: 1.5039 - accuracy: 0.4180 - val_loss: 1.3083 - val_accuracy: 0.5049
Epoch 7/30
898/898 [==============================] - 157s 174ms/step - loss: 1.4611 - accuracy: 0.4353 - val_loss: 1.2950 -

In [7]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test Accuracy: {test_acc}')

225/225 - 6s - loss: 1.1333 - accuracy: 0.5723 - 6s/epoch - 25ms/step
Test Accuracy: 0.5723042488098145


In [8]:
# Save the trained model
model.save('../../models/mood_cnn_model.h5')